# DATA20001 Deep Learning - Group Project
## Text project

**Due Thursday, December 13, before 23:59.**

The task is to learn to assign the correct labels to news articles.  The corpus contains ~850K articles from Reuters.  The test set is about 10% of the articles. The data is unextracted in XML files.

We're only giving you the code for downloading the data, and how to save the final model. The rest you'll have to do yourselves.

Some comments and hints particular to the project:

- One document may belong to many classes in this problem, i.e., it's a multi-label classification problem. In fact there are documents that don't belong to any class, and you should also be able to handle these correctly. Pay careful attention to how you design the outputs of the network (e.g., what activation to use) and what loss function should be used.
- You may use word-embeddings to get better results. For example, you were already using a smaller version of the GloVE  embeddings in exercise 4. Do note that these embeddings take a lot of memory. 
- In the exercises we used e.g., `torchvision.datasets.MNIST` to handle the loading of the data in suitable batches. Here, you need to handle the dataloading yourself.  The easiest way is probably to create a custom `Dataset`. [See for example here for a tutorial](https://github.com/utkuozbulak/pytorch-custom-dataset-examples).

## Download the data

In [1]:
import os
import torch
from torchvision.datasets.utils import download_url
import zipfile

train_path = 'train/'

dl_file='reuters.zip'
dl_url='https://www.cs.helsinki.fi/u/jgpyykko/'
zip_path = os.path.join(train_path, dl_file)
if not os.path.isfile(zip_path):
    download_url(dl_url + dl_file, root=train_path, filename=dl_file, md5=None)

with zipfile.ZipFile(zip_path) as zip_f:
    zip_f.extractall(train_path)
    #os.unlink(zip_path)

The above command downloads and extracts the data files into the `train` subdirectory.

The files can be found in `train/`, and are named as `19970405.zip`, etc. You will have to manage the content of these zips to get the data. There is a readme which has links to further descriptions on the data.

The class labels, or topics, can be found in the readme file called `train/codes.zip`.  The zip contains a file called "topic_codes.txt".  This file contains the special codes for the topics (about 130 of them), and the explanation - what each code means.  

The XML document files contain the article's headline, the main body text, and the list of topic labels assigned to each article.  You will have to extract the topics of each article from the XML.  For example: 
&lt;code code="C18"&gt; refers to the topic "OWNERSHIP CHANGES" (like a corporate buyout).

You should pre-process the XML to extract the words from the article: the &lt;headline&gt; element and the &lt;text&gt;.  You should not need any other parts of the article.

## Your stuff goes here ...

In [1]:
import pandas as pd

In [2]:
# Read data - short version 3400+ documents
reuters = pd.read_pickle('input/reuters_small.pkl')
print(len(reuters))
reuters[0:5]

3426


,codes,headline,text
0,"[C18, C181, CCAT]",Eureko is latest suitor for French insurer GAN.,"\nEureko, an alliance of six European financia..."
1,"[G15, GCAT]",Reuter EC Report Long-Term Diary for July 28 -...,\n****\nHIGHLIGHTS\n****\nLUXEMBOURG - Luxembo...
2,"[G15, GCAT]",Official Journal contents - OJ L 190 of July 1...,\n*\n(Note - contents are displayed in reverse...
3,"[G15, GCAT]",Official Journal contents - OJ C 221 of July 1...,\n*\n(Note - contents are displayed in reverse...
4,"[G15, GCAT]",Official Journal contents - OJ C 220 of July 1...,\n*\n(Note - contents are displayed in reverse...


In [3]:
# load full data, if produced. 435 MB. NOT in Github currently. 
# You can produce this in about 30 min with preprocess_data.ipynb
reuters = pd.read_pickle('input/reuters_all.pkl')

FileNotFoundError: [Errno 2] No such file or directory: 'input/reuters_all.pkl'

In [3]:
# read classcodes
classcodes= pd.read_csv('input/classcodes.csv')
print(len(classcodes))
classcodes[0:12]

126


,Code,Description
0,1POL,CURRENT NEWS - POLITICS
1,2ECO,CURRENT NEWS - ECONOMICS
2,3SPO,CURRENT NEWS - SPORT
3,4GEN,CURRENT NEWS - GENERAL
4,6INS,CURRENT NEWS - INSURANCE
5,7RSK,CURRENT NEWS - RISK NEWS
6,8YDB,TEMPORARY
7,9BNX,TEMPORARY
8,ADS10,CURRENT NEWS - ADVERTISING
9,BNW14,CURRENT NEWS - BUSINESS NEWS


### string-to-int and int-to-string dictionaries for classcodes, turn classes into integers

In [4]:
# add index field to DataFrame
classcodes = classcodes.reset_index()

In [5]:
# Create dictionary index/int to classcode and classcode to int
itocode = dict(zip(classcodes.index, classcodes.Code))
codetoi = dict(zip(classcodes.Code, classcodes.index))

In [1]:
list(itocode.items())[0:7]

NameError: name 'itocode' is not defined

In [7]:
print(itocode[3])
print(codetoi['4GEN'])

4GEN
3


In [8]:
# Turn one list of codes into ints
def listToInt(mylist):
    return [codetoi[item] for item in mylist]

#test
listToInt(['C18', 'C181', 'CCAT'])

[25, 26, 44]

In [9]:
# for each list of codes, turn it to ints
reuters['classes'] = [listToInt(codelist) for codelist in reuters.codes]
reuters[0:3]

,codes,headline,text,classes
0,"[C18, C181, CCAT]",Eureko is latest suitor for French insurer GAN.,"\nEureko, an alliance of six European financia...","[25, 26, 44]"
1,"[G15, GCAT]",Reuter EC Report Long-Term Diary for July 28 -...,\n****\nHIGHLIGHTS\n****\nLUXEMBOURG - Luxembo...,"[80, 90]"
2,"[G15, GCAT]",Official Journal contents - OJ L 190 of July 1...,\n*\n(Note - contents are displayed in reverse...,"[80, 90]"


### Create Y variable as multi-hot

In [10]:
# Multihot, for single list - one row
def multihot(tags):
    return [1 if tag in tags else 0 for tag in taglist]

# list of classes, 126 int: [0...125]
taglist = list(classcodes.index)
#test
multihot([1,3])[0:10]

[0, 1, 0, 1, 0, 0, 0, 0, 0, 0]

In [11]:
# run for each row
Y_hot = [multihot(claslist) for claslist in reuters.classes]

In [12]:
len(Y_hot[0])

126

In [15]:
# headlines, texts in tokenized form
import nltk
nltk.download('punkt')

headlines = [nltk.word_tokenize(row[1]) for _, row in reuters.iterrows()]
texts = [nltk.word_tokenize(row[2]) for _, row in reuters.iterrows()]

[nltk_data] Downloading package punkt to /home/abc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
# collect all distinct words present in headlines & texts
import itertools
from collections import Counter
vocabulary = list(Counter([word for text in itertools.chain(headlines, texts) for word in text]).keys())

In [17]:
# Create dictionary index/int to word and word to int
word_to_code = dict(zip(vocabulary, range(len(vocabulary))))
print(word_to_code['are'])

# headlines & texts in integer format
headlines = [[word_to_code[word] for word in headline] for headline in headlines]
texts = [[word_to_code[word] for word in text] for text in texts]

3317


In [18]:
headlines[1010]

[2963, 1662, 2964, 224, 2965, 2919, 2966, 8]

## Save your model

It might be useful to save your model if you want to continue your work later, or use it for inference later.

In [ ]:
torch.save(model.state_dict(), 'model.pkl')

The model file should now be visible in the "Home" screen of the jupyter notebooks interface.  There you should be able to select it and press "download".

## Download test set

The testset will be made available during the last week before the deadline and can be downloaded in the same way as the training set.

## Predict for test set

You will be asked to return your predictions a separate test set.  These should be returned as a matrix with one row for each test article.  Each row contains a binary prediction for each label, 1 if it's present in the image, and 0 if not. The order of the labels is the order of the label (topic) codes.

An example row could like like this if your system predicts the presense of the second and fourth topic:

    0 1 0 1 0 0 0 0 0 0 0 0 0 0 ...
    
If you have the matrix prepared in `y` you can use the following function to save it to a text file.

In [ ]:
np.savetxt('results.txt', y, fmt='%d')